In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english",output_attentions=True)
from datasets import load_dataset

ds = load_dataset("papluca/language-identification")
def calculate_AE_matrix(attention_matrix):
    return (torch.log(attention_matrix) * attention_matrix * (-1)).sum(dim=3).mean(dim=2)

# 函数原型如下，这里的if条件判断则是为了防止gpu上放置数据过多，导致核崩溃
# 这里需要基于这个函数来微微调整得到各个模型的AE矩阵计算的代码

def get_AE_matrix(model,dataset,heads_per_layer,layers):
    model.to('cpu')
    attention_entropy=torch.tensor([[0.]*heads_per_layer]*layers)
    AE_matrix = torch.tensor([[0.]*heads_per_layer]*layers)
    data_amount = 0
    for data in tqdm(dataset):
        if data_amount/len(dataset)<=0.3:
            if model.device!=torch.device(type='cuda',index=0):
                model.to('cuda')
            attention_entropy=attention_entropy.to('cuda')
            input_text = data['text']
            inputs = tokenizer(input_text,return_tensors='pt').to('cuda')
            output = model(**inputs)
            
            attentions = output[1]
            attentions = torch.cat([(layer) for layer in attentions])
            attentions.to('cuda')
            AE = calculate_AE_matrix(attentions)
            AE.to('cuda')
            attention_entropy += AE
            data_amount+=1
        else:
            model.to('cpu')
            attention_entropy=attention_entropy.to('cpu')
            input_text = data['text']
            inputs = tokenizer(input_text,return_tensors='pt').to('cpu')
            output = model(**inputs)
            attentions = output[1]
            attentions = torch.cat([(layer) for layer in attentions])
            attention_entropy += calculate_AE_matrix(attentions.to('cpu'))
            data_amount+=1
    AE_matrix = attention_entropy / data_amount    
    return AE_matrix

from tqdm import tqdm
import torch
import torch.utils.data as Data

class MyDataset(Data.Dataset):
    def __init__(self,dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset['labels'])
    def __getitem__(self,idx):
        keys = list(self.dataset.keys())
        values = list(self.dataset.values())
        return {keys[0]:values[0][idx],keys[1]:values[1][idx]}

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cpu")


In [3]:
from transformers import BertTokenizer, BertModel
from bertviz import model_view

sentence = 'I am a good student.'
inputs = tokenizer(sentence, return_tensors='pt')

# 获取模型输出（包括注意力权重）
outputs = model(**inputs)

# 可视化注意力
attention = outputs.attentions
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
model_view(attention, tokens)


<IPython.core.display.Javascript object>